## What if we account for seasonal variations in usage and generation?

In [1]:
import datetime
import pandas as pd
import yaml

In [2]:
with open("../data/ballpark_figures.yaml", "r", encoding="utf-8") as filepath:
    parameters = yaml.load(filepath, Loader=yaml.FullLoader)

ballpark_installation_cost = parameters["installation_cost"]
ballpark_sell_back_rate = parameters["sell_back_rate"]
expected_unit_rate = parameters["expected_unit_rate"]
solar_panel_total_output = parameters["solar_panel_total_output"]
solar_panel_lifetime = parameters["solar_panel_lifetime"]

In [3]:
monthly_sunshine_df = pd.read_csv("../data/monthly_sunshine_hours.csv")
meter_readings_df = pd.read_csv("../data/meter_readings.csv")

monthly_sunshine_df["solar_panel_output"] = monthly_sunshine_df.sunshine.apply(lambda x: x*solar_panel_total_output)

In [4]:
meter_readings_df["month"] = meter_readings_df["date"].apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y").month)

monthly_usage_df = meter_readings_df.groupby("month", as_index=False)["weekly_usage"].sum()
monthly_usage_df.rename({"weekly_usage": "monthly_usage"}, axis=1, inplace=True)

In [5]:
# September reading was taken on the 17th.
monthly_usage_df["average_daily_usage"] = monthly_usage_df["monthly_usage"] / [30, 31, 30, 31, 31, 17]

In [6]:
monthly_usage_df["month"] = ["April", "May", "June", "July", "August", "September"]

In [7]:
monthly_sunshine_df["average_daily_output"] = monthly_sunshine_df["solar_panel_output"] / [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

monthly_sunshine_df

,month,sunshine,solar_panel_output,average_daily_output
0,January,42.70,187.880,6.060645
1,February,66.93,294.492,10.517571
2,March,101.15,445.060,14.356774
3,April,148.19,652.036,21.734533
4,May,183.30,806.520,26.016774
5,June,150.13,660.572,22.019067
6,July,136.14,599.016,19.323097
7,August,136.15,599.060,19.324516
8,September,112.91,496.804,16.560133
9,October,85.41,375.804,12.122710


In [21]:
def get_expected_saving(monthly_output, monthly_usage, unit_rate=0.2982, sell_back_rate=0.18):
    if monthly_usage == 0 or monthly_usage != monthly_usage:
        return None
    leftover_units = monthly_output - monthly_usage
    if leftover_units >= 0:
        return (monthly_usage * unit_rate) + leftover_units * sell_back_rate
    return monthly_output * unit_rate

In [22]:
output_and_usage = pd.merge(monthly_sunshine_df, monthly_usage_df, how="outer", on="month")
output_and_usage.loc[output_and_usage["month"] == "September", "monthly_usage"] = 720
output_and_usage["expected_saving_0.2982"] = output_and_usage.apply(
    lambda row: get_expected_saving(row["solar_panel_output"], row["monthly_usage"]), axis=1
)

output_and_usage["expected_saving_0.15"] = output_and_usage.apply(
    lambda row: get_expected_saving(row["solar_panel_output"], row["monthly_usage"], unit_rate=0.15, sell_back_rate=0.08), axis=1
)



In [23]:
output_and_usage

,month,sunshine,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage,expected_saving_0.2982,expected_saving_0.15
0,January,42.70,187.880,6.060645,NaN,NaN,NaN,NaN
1,February,66.93,294.492,10.517571,NaN,NaN,NaN,NaN
2,March,101.15,445.060,14.356774,NaN,NaN,NaN,NaN
3,April,148.19,652.036,21.734533,0.0,0.000000,NaN,NaN
4,May,183.30,806.520,26.016774,408.0,13.161290,193.399200,93.08160
5,June,150.13,660.572,22.019067,379.0,12.633333,163.700760,79.37576
6,July,136.14,599.016,19.323097,450.0,14.516129,161.012880,79.42128
7,August,136.15,599.060,19.324516,483.0,15.580645,164.921400,81.73480
8,September,112.91,496.804,16.560133,720.0,24.000000,148.146953,74.52060
9,October,85.41,375.804,12.122710,NaN,NaN,NaN,NaN


In [ ]:
# TODO: Economy 7 - we can estimate what proportion of bought units might be bought with night tariff vs day tariff (in practice not sure if this level of intelligence is possible)